# <span style="color:darkblue"> Lecture 20 - Connecting to SQL via Python </span>

<font size = "5">

In the past lectures:

- We worked directly in SQL
- Imported data
- Carried out basic dataset operations

In  this lecture we will run SQL from Python

- Combine the best of both!
- Learn about escape characters!

# <span style="color:darkblue"> I. Import Libraries and Data </span>


<font size = "5">

Install new libraries to connect to SQL from Python:

- psycopg2
- sqlalchemy

In [1]:
# With base Python, run the following commands in the terminal
# (i)     pip3 install psycopg2
# (ii)    pip3 install sqlalchemy
# If (i) doesn't work, instead try
#         pip3 install psycopg2-binary           OR
#         pip3 install psycopg2-binary --user

# For Anaconda: Open the "Anaconda Navigator" app and go the "Environment" tab
# Check whether "psycopg2", "sqlalchemy" appear in the "installed" panel
# Otherwise, search in the "Not installed" panel and install them

<font size = "5">

Import libraries

In [2]:
# psycogpg2 helps us process SQL commands to send to the server
# sqlalchemy facilitates establishing a connection with the server

import pandas as pd
import psycopg2
import sqlalchemy as sa
from   sqlalchemy.engine import URL
from   sqlalchemy import text

<font size = "5">

Import Data

In [3]:
# Import the two car racing datasets

results  = pd.read_csv("data_raw/results.csv")
races    = pd.read_csv("data_raw/races.csv")
circuits = pd.read_csv("data_raw/circuits.csv")

<font size = "5">

Connect to SQL server

In [4]:
# Use the connection details to your server
# These are the default settings.
# "postgresql" is a fixed argument
# If you have a different host,database, username, or password,
# change the corresponding connection details

url_server = URL.create(
    "postgresql",
    host = 'localhost',
    database = 'postgres',
    username = 'postgres',
    port = 5432,
    password  = '1234')

con = sa.create_engine(url_server).connect()


<font size = "5">

Upload data to SQL

In [5]:
# Import the "results" table, with the name "results_sql"
# Con is an argument to specify the server connection
# "if_exists" is an option to replace the table if it already exists
# You can choose any name instead of "results_sql"

results.to_sql('results_sql',
               con = con,
               if_exists='replace')

# Import "races"
races.to_sql('races_sql',con, if_exists='replace')

102

<font size = "5">

Try it yourself!

- Upload the "circuits table" into SQL <br>
using ".to_sql()"

In [6]:
# Write your own code

circuits.to_sql('circuits_sql',
                 con = con, 
                 if_exists = 'replace')



77

# <span style="color:darkblue"> II. "Escape" Characters </span>

<font size = "5">

- Backslash (\\) is known as an escape character
- Used for special operations on strings
- Great for working with text!

<font size = "5">

Multi-line strings (" \ ")

In [7]:
# Use a backslash to define strings over multiple lines
# Do not include a space after "\", otherwise it won't work.
example_string = "This is a string \
                  defined over multiple lines"

print(example_string)

This is a string                   defined over multiple lines


<font size = "5">

"Double quotes" inside string

In [8]:
# Use a backslash + quotation 

example_double = "This will \"supposedly\" put double quotes inside a string"
print(example_double)

This will "supposedly" put double quotes inside a string


<font size = "5" >
'Single quotes' inside string

In [9]:
# There is no need for a backslash given single quotes 

example_single = "This will 'supposedly' put single quotes inside a string"
print(example_single)

This will 'supposedly' put single quotes inside a string


<font size = "5">

Try it yourself!

- Print a string ``` SELECT "driverId" FROM results; ``` <br> 
using backslash

In [10]:
# Write your own code

example_string2 = "SELECT \"driverId\" FROM results;"
print(example_string2)



SELECT "driverId" FROM results;


# <span style="color:darkblue"> II. Data operations </span>

<font size = "5">

- In SQL, operations are often called "queries"


<font size = "5">

Import Data from SQL

In [11]:
# Use "pd.read_sql()"
# - The first argument is string with instructions wrapped in text()
#- The second argument is the server connection

# Extract all data from a column
example1 = pd.read_sql(text("SELECT * FROM results_sql;"),con)

# Extract a subset of columns, in this case the points column 
example2 = pd.read_sql(text("SELECT points \
                             FROM results_sql;"),con)

# Subset based on a string condition
example3 = pd.read_sql(text("SELECT * \
                             FROM races_sql \
                             WHERE name = 'Abu Dhabi Grand Prix';"),con)



# Note: (i) Remember to include "\"  to be able to define
#           strings over multiple lines
#       (ii) We can include single quotations in the WHERE command
#            without any additional escape characters

<font size = "5">

Upper case columns

- In SQL syntax we use double quotes e.g. ``` "driverId" ```
- ```.read_sql()``` requires a string inside a string
- To do so, use escape characters, e.g.   ``` \"driverId\" ```

In [12]:
# Select a column
example4 = pd.read_sql(text("SELECT \"driverId\" \
                             FROM results_sql;"),con)

# Compute an aggregate statistic
example5 = pd.read_sql(text("SELECT AVG(points) as mean_points \
                             FROM results_sql \
                             GROUP BY \"driverId\" ;"),con)


<font size = "5">

Merge two datasets

In [13]:
# Merge 
# Reference the column \"driverId\" with escape characters
example6 = pd.read_sql(text("SELECT * \
                             FROM results_sql \
                             LEFT JOIN races_sql \
                             ON results_sql.\"raceId\" = races_sql.\"raceId\" ;"),con)

# Merge a subset of columns
# Use "results_sql.*" to select all columns from the primary dataset
# Use "races_sql.date" to only select the "date" column from the secondary dataset 

example7 = pd.read_sql(text("SELECT results_sql.*, races_sql.date \
                             FROM results_sql \
                             LEFT JOIN races_sql \
                             ON results_sql.\"raceId\" = races_sql.\"raceId\" ;"),con)
display(example6)

,index,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,...,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,0,1,18,1,1,22,1,1,1,1,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,1,2,18,2,2,3,5,2,2,2,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,2,3,18,3,3,7,7,3,3,3,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,3,4,18,4,4,5,11,4,4,4,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,4,5,18,5,1,23,3,5,5,5,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,25835,25841,1096,854,210,47,12,16,16,16,...,2022-11-18,10:00:00,2022-11-18,13:00:00,2022-11-19,11:00:00,2022-11-19,14:00:00,\N,\N
25836,25836,25842,1096,825,210,20,16,17,17,17,...,2022-11-18,10:00:00,2022-11-18,13:00:00,2022-11-19,11:00:00,2022-11-19,14:00:00,\N,\N
25837,25837,25843,1096,1,131,44,5,18,18,18,...,2022-11-18,10:00:00,2022-11-18,13:00:00,2022-11-19,11:00:00,2022-11-19,14:00:00,\N,\N
25838,25838,25844,1096,849,3,6,20,19,19,19,...,2022-11-18,10:00:00,2022-11-18,13:00:00,2022-11-19,11:00:00,2022-11-19,14:00:00,\N,\N


<font size = "5">

Try it yourself!

- Practice the ```pd.read_sql()``` command
- FROM results_sql compute the sum of points by <br>
"raceId" <br>

In [14]:
# Write your own code
my_example8 = pd.read_sql(text("SELECT SUM(points)\
                                FROM results_sql\
                                GROUP BY  results_sql.\"raceId\" =  " ) ,con)

display ( my_example8)





ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at end of input
LINE 1: ...                          GROUP BY  results_sql."raceId" =  
                                                                       ^

[SQL: SELECT SUM(points)                                FROM results_sql                                GROUP BY  results_sql."raceId" =  ]
(Background on this error at: https://sqlalche.me/e/14/f405)

<font size = "5">

Try it yourself!

- Practice the ```pd.read_sql()``` command
- Merge "races_sql" and the circuits table on "circuitId" <br>
tha you imported above using ```LEFT JOIN```

In [15]:
# Write your own

example9 = pd.read_sql(text("SELECT *\
                            FROM races_sql\
                            LEFT JOIN circuits_sql\
                            ON races_sql.\"circuitId\" = circuits_sql.\"circuitId\" ; "), con)


display(example9)

,index,raceId,year,round,circuitId,name,date,time,url,fp1_date,...,index,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1081,1100,2023,3,1,Australian Grand Prix,2023-04-02,05:00:00,https://en.wikipedia.org/wiki/2023_Australian_...,2023-03-31,...,0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,1059,1076,2022,3,1,Australian Grand Prix,2022-04-10,05:00:00,http://en.wikipedia.org/wiki/2022_Australian_G...,2022-04-08,...,0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
2,997,1010,2019,1,1,Australian Grand Prix,2019-03-17,05:10:00,http://en.wikipedia.org/wiki/2019_Australian_G...,\N,...,0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
3,976,989,2018,1,1,Australian Grand Prix,2018-03-25,05:10:00,http://en.wikipedia.org/wiki/2018_Australian_G...,\N,...,0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
4,956,969,2017,1,1,Australian Grand Prix,2017-03-26,05:00:00,http://en.wikipedia.org/wiki/2017_Australian_G...,\N,...,0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1056,1072,2021,21,77,Saudi Arabian Grand Prix,2021-12-05,17:30:00,http://en.wikipedia.org/wiki/2021_Saudi_Arabia...,2021-12-03,...,74,77,jeddah,Jeddah Corniche Circuit,Jeddah,Saudi Arabia,21.6319,39.1044,15,http://en.wikipedia.org/wiki/Jeddah_Street_Cir...
1098,1096,1115,2023,18,78,Qatar Grand Prix,2023-10-08,14:00:00,https://en.wikipedia.org/wiki/2023_Qatar_Grand...,2023-10-06,...,75,78,losail,Losail International Circuit,Al Daayen,Qatar,25.4900,51.4542,\N,http://en.wikipedia.org/wiki/Losail_Internatio...
1099,1038,1051,2021,20,78,Qatar Grand Prix,2021-11-21,14:00:00,http://en.wikipedia.org/wiki/2021_Qatar_Grand_...,2021-11-19,...,75,78,losail,Losail International Circuit,Al Daayen,Qatar,25.4900,51.4542,\N,http://en.wikipedia.org/wiki/Losail_Internatio...
1100,1083,1102,2023,5,79,Miami Grand Prix,2023-05-07,19:30:00,https://en.wikipedia.org/wiki/2023_Miami_Grand...,2023-05-05,...,76,79,miami,Miami International Autodrome,Miami,USA,25.9581,-80.2389,\N,http://en.wikipedia.org/wiki/Miami_Internation...


# <span style="color:darkblue"> IV. Additional Material </span>

<font size = "5">

More about SQL syntax

https://www.w3schools.com/sql/